In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# Pre-filtering with Chroma DB to avoid GIGO.


# 1) 메타데이터는 어떻게 사용되는걸까?
# indexing을 하며 계속 의문점이 들었다. 대체 메타데이터를 어떻게 활용하는거지? 머릿속에 든 의문점을 하나씩 적어보자.
# 1. 기본적으로 프롬프팅 시 메타데이터를 포함하여 프롬프팅하므로, LLM은 메타데이터를 참고하긴 한다.
# 2. 예를 들어, 연도 정보가 중요한 상황일 때, LLM은 충분히 똑똑하므로 현재 시간과 연도 메타데이터를 참고하여 자체적으로 연도가 지난 데이터를 참고하지 않는다.
# 3. 즉 r 과정을 통해 가져온 정보가 정확하지 않다면.. 예를 들어 k가 3일 때, 3개 청크의 메타데이터 전부가 2020년의 정보라면, LLM은 지원 가능한 정책이 없다 등으로 답변할 것이다.
# 4. 여기서 pre-filtering의 중요성이 대두된다. GIGO를 피하기 위해.

# 2) R의 근본적 한계
# 1. r은 벡터 DB가 지원하는 단순 유사도 검색을 통해 그 결과를 가져온다.
# 2. 즉, 개발자가 필터링 조건을 걸지 않는 이상, 질문과 가장 유사한 벡터를 가져온다. 그것의 메타데이터가 어떻든지 상관하지 않고.
# 3. 예를 들어 '30대 남성이 지원 가능한 미술 정책 검색해줘' 라는 질문을 받는다면, DB에 존재하는 모든 문서들.. 00년 부터 25년까지.. 단순 저 질문 자체의 벡터와 가장 유사한 것들을 가져온다는 말이다.
# 4. 어쩌면 25년 정책으로 청크들이 구성될 수도 있겠지만, 항상 이를 보장하지는 않는다는 것이 문제이다.
# 5. 따라서 이 바보같지만 착한 리트리버를 위해 우리가 필터링을 해 주어야 한다는 것이다(pre-filtering)
# 6. 다행히도 벡터DB들은 필터링 쿼리를 제공하므로, 우리는 정규식이나 코드 또는 sLLM 등을 통해 사용자 질문을 기반으로 벡터 쿼리를 잘 생성하기만 하면 된다.



In [ ]:
# Chroma DB 세팅하기
# https://docs.trychroma.com/docs/querying-collections/metadata-filtering

# Chroma는 두 가지 유형의 필터를 제공.
# 원하는 조건을 dict 형태로 제공한다.

# 메타데이터 필터: Collection.query() 또는 Collection.get()에서 where 절을 사용하여 메타데이터를 기반으로 문서를 필터링
# 문서 필터: Collection.query() 또는 Collection.get()에서 where_document를 사용하여 문서 내용을 기반으로 필터링

# 필터 연산자(예: $eq, $gt)등을 사용하여 명시적인 필터링이 가능하다.
# 이때, 메타데이터의 타입을 구분하여 처리하므로, 숫자 비교 연산자를 올바르게 사용하려면, 메타 데이터의 값도 숫자(int, float) 타입이어야 한다.
# 따라서 메타 데이터 필터링을 고려한다면 단순 문자열 코딩은 피해야 한다.
# 날짜도 마찬가지.

# 5.1 이상의 값
# results = collection.query(
#     query_texts=["검색할 문서입니다"],
#     n_results=2,
#     where={"rating": {"$gte": 5.1}}
# )

# 크로마 쿼리 작성도 llm이 잘하네!

In [ ]:
# 따라서 이전의 document from csv 함수를 좀 수정하자
# 단순 문자열 필드 -> 숫자 필드로. 날짜 또한 timestamp보단 YYMMDD의 숫자로.

import csv
from langchain_core.documents import Document
from datetime import datetime

def create_documents_from_csv(csv_file_path: str) -> list[Document]:
    """
    CSV 파일에서 정책 정보를 읽어와 LangChain의 Document 객체 리스트로 변환합니다.
    (날짜 처리: YYYYMMDD 정수 변환 방식 적용)
    """
    
    documents = []
    
    try:
        with open(csv_file_path, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            
            for row in reader:
                # 1. page_content 구성
                page_content = f"""
정책명: {row.get('plcyNm', '정보 없음')}
정책 설명: {row.get('plcyExplnCn', '정보 없음')}
지원 내용: {row.get('plcySprtCn', '정보 없음')}
신청 자격: {row.get('addAplyQlfcCndCn', '정보 없음')}
"""
                # 2. metadata 기본 구성
                metadata = {
                    "policy_id": row.get('plcyNo'),
                    "policy_name": row.get('plcyNm'),
                    "main_department": row.get('sprvsnInstCdNm'),
                    "operating_department": row.get('operInstCdNm'),
                    "category_large": row.get('lclsfNm'),
                    "category_medium": row.get('mclsfNm'),
                }

                # 3. 숫자 필드 타입 변환
                numeric_fields = {
                    "min_age": row.get('sprtTrgtMinAge'),
                    "max_age": row.get('sprtTrgtMaxAge'),
                }
                for key, value in numeric_fields.items():
                    if value is not None and value.strip() != '':
                        try:
                            metadata[key] = int(value)
                        except ValueError:
                            metadata[key] = value

                # 4. 💡 날짜 필드를 YYYYMMDD 정수로 변환
                application_period_str = row.get('aplyYmd', '')
                if ' ~ ' in application_period_str:
                    try:
                        start_date_str, end_date_str = application_period_str.split(' ~ ')
                        
                        # 하이픈(-)을 제거하고 정수로 변환 (예: "2025-07-21" -> 20250721)
                        metadata['start_date_int'] = int(start_date_str.strip().replace('-', ''))
                        metadata['end_date_int'] = int(end_date_str.strip().replace('-', ''))
                        
                        metadata['application_period_str'] = application_period_str # 원본 문자열도 저장
                    except (ValueError, TypeError):
                        metadata['application_period_raw'] = application_period_str
                else:
                     metadata['application_period_raw'] = application_period_str

                # 5. 최종 Document 생성
                cleaned_metadata = {k: v for k, v in metadata.items() if v is not None and v != ''}
                documents.append(Document(page_content=page_content.strip(), metadata=cleaned_metadata))
                
    except FileNotFoundError:
        print(f"오류: '{csv_file_path}' 파일을 찾을 수 없습니다.")
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        
    return documents

In [ ]:
# 테스트
data = create_documents_from_csv("./policy_data.csv")
# data = create_documents_from_csv("./test.csv")
data

In [ ]:
# 청킹(kss)
# 2. kss 청킹

import kss


def split_docs_with_kss(documents: list[Document], chunk_size: int = 500) -> list[Document]:
    """
    kss를 사용해 문서를 문장 단위로 분할하고,
    정해진 chunk_size에 맞게 문장들을 다시 그룹화합니다.
    """
    final_chunks = []
    
    for doc in documents:
        # 1. kss로 전체 page_content를 문장 리스트로 분리
        sentences = kss.split_sentences(doc.page_content)
        
        current_chunk_content = ""
        # 2. 문장들을 순회하며 chunk_size에 가깝게 그룹으로 묶기
        for sentence in sentences:
            # 현재 문장을 추가하면 chunk_size를 초과하는지 확인
            if len(current_chunk_content) + len(sentence) > chunk_size and current_chunk_content:
                # 초과한다면, 현재까지의 내용을 하나의 chunk로 만듦
                final_chunks.append(
                    Document(page_content=current_chunk_content.strip(), metadata=doc.metadata)
                )
                # 새로운 chunk 시작
                current_chunk_content = sentence
            else:
                # 초과하지 않으면, 현재 chunk에 문장 추가
                current_chunk_content += " " + sentence

        # 마지막에 남아있는 chunk를 리스트에 추가
        if current_chunk_content:
            final_chunks.append(
                Document(page_content=current_chunk_content.strip(), metadata=doc.metadata)
            )
            
    return final_chunks


In [ ]:
split_data = split_docs_with_kss(data)
# --- 결과 확인 (실제 실행 시 주석 해제) ---
print(f"원본 Document 수: {len(data)}")
print(f"분할된 Chunk 수 (kss): {len(split_data)}")

if split_data:
    print("\n--- 첫 번째 분할된 Chunk (kss) ---")
    print(split_data[0].page_content)
    print(split_data[0].metadata)

In [ ]:
# Chroma DB 세팅
# Chroma DB는 FAISS와 달리, 정말 벡터 DB 라이브러리이다.
# 먼저 어떤 임베딩 모델을 사용할지 정하고, 해당 임베딩 모델과 연결된 컬렉션(collection)을 생성해야 한다.

from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings


# # 1. 사용할 임베딩 모델 선택 및 초기화
# # 예시: 로컬에서 실행하는 Ollama 모델 사용
# embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# # 2. ChromaDB 컬렉션 생성 (from_documents 메서드 사용)
# # 이 과정에서 LangChain이 documents의 page_content를 임베딩하고, metadata와 함께 저장합니다.
# vectorstore = Chroma.from_documents(
#     documents=split_data,
#     embedding=embedding_model,
#     collection_name="policy_collection", # 컬렉션 이름 지정
#     persist_directory="./chroma_db" # DB를 저장할 경로
# )

# print("ChromaDB에 Document 저장 완료!")

# # 위 과정을 수행하며 이런 오류를 마주쳤다
# BadRequestError: Error code: 400 - {'error': {'message': 'Requested 414416 tokens, max 300000 tokens per request', 'type': 'max_tokens_per_request', 'param': None, 'code': 'max_tokens_per_request'}}
# 이는 임베딩 모델의 API 호출 한도를 초과하여 발생하는 오류로, 비어있는 벡터 DB를 생성 후 반복문을 통해 임베딩하는 것으로 해결해야 한다고 한다.



In [ ]:
# 데이터 처리
def add_to_chroma_in_batches(docs: list[Document], batch_size: int = 100):
    """문서 리스트를 배치로 나누어 ChromaDB에 추가합니다."""
    
    # 전체 문서 리스트를 batch_size만큼 건너뛰며 반복
    for i in range(0, len(docs), batch_size):
        # 현재 처리할 배치 슬라이싱
        batch = docs[i:i + batch_size]
        
        # 현재 배치만 DB에 추가
        vectorstore.add_documents(documents=batch)
        
        # 진행 상황 출력
        print(f"Batch {i//batch_size + 1}/{(len(docs) - 1)//batch_size + 1} 처리 완료 ({len(batch)}개 문서 추가)")

# 함수 호출로 배치 처리 실행
add_to_chroma_in_batches(data, batch_size=200) # 배치 사이즈는 조절 가능

print("\n모든 문서의 배치 처리가 완료되었습니다.")



In [ ]:
# 데이터가 잘 저장되었는지 확인
# vectorstore.persist()
# .persist()가 이전 버전에서 필요했던 이유는 쓰기 작업이 강제로 플러시될 때만 수행되었기 때문입니다. Chroma 0.4.0은 모든 쓰기를 즉시 디스크에 저장하므로 persist가 더 이상 필요하지 않습니다.

retriever = vectorstore.as_retriever()
results = retriever.invoke("예술 관련 정책 찾아줘")
print(results)

In [ ]:
# 조건 쿼리 테스트
# 연도, 나이 범위로 먼저 필터링 해보자!

In [ ]:
# --- 시나리오 1: 특정 카테고리 필터링 ---
# '일자리' 분야의 정책 중에서 '청년 지원'과 관련된 내용 검색
print("--- [시나리오 1] 카테고리 필터링: '일자리' 분야 ---")
results_category = vectorstore.similarity_search(
    query="청년 지원",
    k=5,
    filter={"category_large": "일자리"} # category_large 메타데이터가 '일자리'인 문서만 대상
)
for doc in results_category:
    print(f"  - 정책명: {doc.metadata.get('policy_name', 'N/A')}")
    print(f'내용\n {doc.page_content}')
    print("-" * 20)

In [ ]:
# --- 시나리오 2: 나이(숫자 범위) 필터링 ---
# 만 19세 이상 39세 미만 청년을 대상으로 하는 정책 검색
print("\n--- [시나리오 2] 숫자 범위 필터링: 19세 이상 39세 미만 ---")
results_age = vectorstore.similarity_search(
    query="자금 지원",
    k=5,
    filter={
        "$and": [
            {"min_age": {"$gte": 19}}, # 'min_age'가 19보다 크거나 같고
            {"max_age": {"$lt": 39}}   # 'max_age'가 39보다 작은 문서
        ]
    }
)
for doc in results_age:
    print(f"  - 정책명: {doc.metadata.get('policy_name', 'N/A')}, 연령: {doc.metadata.get('min_age')}~{doc.metadata.get('max_age')}")
    print(f'내용\n {doc.page_content}')
    print("-" * 20)


In [ ]:
# --- 시나리오 3: 숫자형 날짜(YYMMDD) 필터링
today_int = int(datetime.now().strftime("%Y%m%d"))
print(f"오늘 날짜(정수형): {today_int}\n")

# 3. '현재 신청 가능한' 정책 필터링 검색
print("--- 현재 신청 가능한 정책 검색 (정수 비교) ---")
results_ongoing = vectorstore.similarity_search(
    query="주거 지원",
    k=3,
    filter={
        "$and": [
            {"start_date_int": {"$lte": today_int}},
            {"end_date_int": {"$gte": today_int}}
        ]
    }
)

if not results_ongoing:
    print("현재 신청 가능한 정책이 없습니다.")
else:
    for doc in results_ongoing:
        print(f"  - 정책명: {doc.metadata.get('policy_name', 'N/A')}")
        print(f"    신청 기간: {doc.metadata.get('application_period_str', 'N/A')}")

In [ ]:
# --- 시나리오 4: 복합 조건 필터링 ($or 사용) ---
# '보건복지부' 또는 '여성가족부'에서 주관하며, '주거' 또는 '금융' 분야인 정책 검색
print("\n--- [시나리오 4] 복합 조건 필터링: 부처 및 분야 동시 검색 ---")
results_complex = vectorstore.similarity_search(
    query="저소득층 지원",
    k=5,
    filter={
        "$and": [
            {
                "$or": [
                    {"main_department": "보건복지부"},
                    {"main_department": "여성가족부"}
                ]
            },
            {
                "$or": [
                    {"category_large": "주거"},
                    {"category_large": "금융"}
                ]
            }
        ]
    }
)
for doc in results_complex:
    print(f"  - 정책명: {doc.metadata.get('policy_name', 'N/A')}, 주관부처: {doc.metadata.get('main_department')}, 분야: {doc.metadata.get('category_large')}")
print("-" * 20)

In [ ]:
# 이제 이렇게 조건 기반 쿼리를 사용할 준비가 되었다면..
# 사용자의 질문을 가공하여 쿼리로 변환해야 한다.
# 대표적으로 규칙기반(정규식 등)과 LLM을 활용한 방법이 있다.

# 규칙기반 시스템의 경우..
# 장점 (Pros)	단점 (Cons)
# ✅ 매우 빠름 (LLM API 호출 없음)	⚠️ 유연하지 않음 ("내년이면 서른" 같은 표현 처리 불가)
# ✅ 예측 가능하고 일관적임	⚠️ 규칙이 복잡해지면 유지보수가 어려움 (if/elif 지옥)
# ✅ 비용이 들지 않음	⚠️ 새로운 패턴에 취약함

# LLM을 활용할 경우..(자연어 이해-NLU)
# 장점 (Pros)	단점 (Cons)
# ✅ 매우 유연하고 강력함 (문맥, 동의어, 복잡한 문장 이해)	⚠️ 느림 (API 네트워크 지연 시간)
# ✅ 유지보수가 쉬움 (프롬프트만 수정하면 됨)	⚠️ 비용 발생 (API 호출 비용)
# ✅ 다양한 사용자 표현에 강함	⚠️ 결과가 100% 일관적이지 않을 수 있음

# 최고의 시스템 및 성능을 원한다면 하이브리드 고려. 
# 하이브리드는 간단하게나마 우리 시스템에도 적용할 수 있을 것 같음.
# 모든 입력에 대해 LLM으로 처리한다면, 많은 비용이 들 것. 
# 즉, 대놓고 이상한 요청은 규칙 기반으로 충분히 처리 가능하므로.. 이를 기반으로 구축.

In [ ]:
# 단계별 필터링 구조(하이브리드)

# 1단계) 사전 차단(Guardrail)
# 대상 : 인사말, 욕설/비속어, 무의미한 입력, 주제와 무관한 질문 등
# 처리 : 미리 준비된 답변을 반환. (예: 안녕하세요! 청년 정책 정보를 찾아드리는 챗 봇입니다!)

# 2단계) 간단한 규칙 기반 추출(Fast Path)
# 대상 : 키워드나 숫자가 명확히 드러나는 질문(예: 30살 일자리 정책)
# 처리 : 규칙 기반 함수 등을 통해 즉시 filter 생성 후 벡터 DB 쿼리.

# 3단계) LLM 호출(Fallback)
# 대상 : 복잡한 자연어 질문
# 처리 : LLM을 호출하여 의도를 파악하고 JSON으로 구조화된 정보 추출. 이를 기반으로 한 filter 생성 후 쿼리
# (질의 분석, 화행 분류, 질의 정제, NL2SQL 등의 과정..)

# 2단계는 생략하고 1, 3단계만 사용하자.

In [ ]:
import os
import json
from openai import OpenAI


In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [ ]:
# 1단계 : 사전 차단
def is_policy_related_query(query: str) -> bool:
    """
    사용자 질문이 정책 추천 도메인과 관련이 있는지 규칙 기반으로 판단합니다.
    (1단계: Guardrail)
    """
    # 긍정 키워드: 이 단어들이 포함되면 관련 질문으로 간주
    policy_keywords = [
        '정책', '지원', '혜택', '대출', '주거', '일자리', '금융', '교육',
        '청년', '신청', '자격', '소득', '보증금', '월세', '취업', '창업'
    ]
    
    # 부정 키워드: 이 단어들이 포함되면 관련 없는 질문으로 간주
    unrelated_keywords = [
        '날씨', '맛집', '노래', '영화', '드라마', '안녕', '반가워', 'ㅎㅇ',
        '사랑', '여행', '게임', '스포츠'
    ]

    query_lower = query.lower()

    # 부정 키워드가 하나라도 있으면 즉시 False 반환
    if any(keyword in query_lower for keyword in unrelated_keywords):
        return False

    # 긍정 키워드가 하나라도 있으면 True 반환
    if any(keyword in query_lower for keyword in policy_keywords):
        return True
        
    # 위 두 조건에 모두 해당하지 않으면, 관련 없는 것으로 간주 (보수적 접근)
    return False


In [ ]:
# 2단계 : 상세 정보 추출
def extract_structured_data_with_llm(query: str) -> dict:
    from datetime import datetime
    current_date_int = int(datetime.now().strftime('%Y%m%d'))
    
    system_prompt = f"""
    당신은 대한민국 청년 정책에 대한 사용자 질문을 분석하여, Chroma 데이터베이스 검색 필터로 사용할 JSON을 생성하는 AI 어시스턴트입니다.
    
    # 기본 원칙:
    - 사용자가 기간을 명시하지 않으면, 반드시 '현재 신청 가능한' 정책을 찾는 것을 기본값으로 삼아야 합니다.
    - 이를 위해, 현재 날짜({current_date_int})를 기준으로 `start_date_int`와 `end_date_int`를 비교하는 필터를 생성해야 합니다.
    - **중요**: Chroma는 최상위 레벨에 정확히 하나의 연산자만 허용하므로, 모든 조건을 `$and` 배열로 감싸야 합니다.
    
    # 지침:
    1. 사용자의 질문을 분석하여 필드 정보를 추출하세요.
    
    2. **필터 구조**: 모든 필터는 반드시 다음과 같은 구조를 따라야 합니다:
       {{"$and": [조건1, 조건2, ...]}}
    
    3. **기본 날짜 필터**: 사용자가 기간을 특정하지 않은 모든 경우, 아래 조건들을 **반드시 포함**하세요:
       {{"start_date_int": {{"$lte": {current_date_int}}}}},
       {{"end_date_int": {{"$gte": {current_date_int}}}}}
    
    4. **나이 필터**: 
       - 특정 나이 범위(예: "20대")를 언급하면, 해당 범위의 정책을 찾기 위해:
         {{"min_age": {{"$lte": 최대연령}}}},  // 정책의 최소 나이가 사용자 최대 나이보다 작거나 같음
         {{"max_age": {{"$gte": 최소연령}}}}   // 정책의 최대 나이가 사용자 최소 나이보다 크거나 같음
       - 예: 20대 → min_age <= 29, max_age >= 20
    
    5. **카테고리 필터**:
       - 단순 일치: {{"category_large": "주거"}}
       - 연산자 사용: {{"category_large": {{"$eq": "주거"}}}}
    
    6. **예외 처리**: 사용자가 '과거', '작년', '모든' 등 기간 제한 없이 검색하길 명시적으로 원하면, 날짜 필터를 포함하지 마세요.
    
    7. 추가 설명 없이 오직 JSON 객체만 응답해야 합니다.
    
    # 최종 출력 JSON 예시:
    
    ## 예시 1: "20대 주거 정책 찾아줘" (기본 필터링 적용)
    {{
        "$and": [
            {{"min_age": {{"$lte": 29}}}},
            {{"max_age": {{"$gte": 20}}}},
            {{"category_large": "주거"}},
            {{"start_date_int": {{"$lte": {current_date_int}}}}},
            {{"end_date_int": {{"$gte": {current_date_int}}}}}
        ]
    }}
    
    ## 예시 2: "현재 신청 가능한 청년 정책" (카테고리 없이)
    {{
        "$and": [
            {{"start_date_int": {{"$lte": {current_date_int}}}}},
            {{"end_date_int": {{"$gte": {current_date_int}}}}}
        ]
    }}
    
    ## 예시 3: "작년에 했던 모든 정책 알려줘" (기간 제한 없음)
    {{}}
    
    ## 예시 4: "25살이 받을 수 있는 교육 지원"
    {{
        "$and": [
            {{"min_age": {{"$lte": 25}}}},
            {{"max_age": {{"$gte": 25}}}},
            {{"category_large": "교육"}},
            {{"start_date_int": {{"$lte": {current_date_int}}}}},
            {{"end_date_int": {{"$gte": {current_date_int}}}}}
        ]
    }}
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o",  # gpt-4o 또는 gpt-4o-mini 추천
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query}
            ],
            temperature=0,
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"LLM 호출 중 오류 발생: {e}")
        return {"error": str(e)}

In [ ]:
# 최종 지휘자(Orchestrator) 함수
def process_query_hybrid(query: str):
    """하이브리드 방식으로 사용자 질문을 처리하여 최종 ChromaDB 필터를 생성합니다."""
    print(f"입력 질문: \"{query}\"")
    
    if not is_policy_related_query(query):
        print("-> [처리 결과: 규칙 기반 차단]")
        return "정책과 관련된 질문을 해주세요."
    
    print("-> [처리 방식: LLM 호출하여 구조화된 데이터 추출]")
    structured_data = extract_structured_data_with_llm(query)
    print(f"   - LLM 추출 결과 (JSON): {structured_data}")
    return structured_data

In [ ]:
print("--- Case 1: 관련 없는 질문 ---")
final_filter1 = process_query_hybrid("오늘 서울 날씨 어때?")

print("--- Case 2: 단일 나이 + 카테고리 질문 ---")
final_filter2 = process_query_hybrid("서른 살인데, 받을 수 있는 주거 정책이 궁금해")

print("--- Case 3: 나이 범위 + 담당부처 질문 ---")
final_filter3 = process_query_hybrid("20대를 위한 고용노동부 일자리 정책 좀 찾아줘")

print("--- Case 4: 정보는 없지만 관련 있는 질문 ---")
final_filter4 = process_query_hybrid("청년들을 위한 지원 정책에는 어떤 종류가 있나요?")


In [ ]:
# # 리트리버 기본 세팅
# retriever = vectorstore.as_retriever(
#     search_type="mmr",
#     search_kwargs={'k':5}
# )

In [ ]:
user_query = "20대 청년이 받을 수 있는 정책이 궁금해"

In [ ]:
ff = process_query_hybrid(user_query)


In [ ]:
# retrieved_docs = retriever.invoke(
#     user_query,
#     search_kwargs={'filter': ff}
# )
# # 필터링이 참 어렵구나.. 로직의 발상 자체에는 문제가 없어도.. 데이터 자체에 문제가 있는 경우가 많아서 ㅠㅠ

# 테스트 필터링
# ff = {
#     "$and": [
#         {"min_age": {"$lte": 20}},  # 주의: 연산자 위치 수정
#         {"max_age": {"$gte": 29}},  # 주의: 연산자 위치 수정
#         {"category_large": {"$eq": "주거"}},
#         {"start_date_int": {"$lte": 20250721}},
#         {"end_date_int": {"$gte": 20250721}}
#     ]
# }



In [ ]:
# 리트리버 대신 벡터 스토어의 유사도 검색 메서드를 직접 호출
retrieved_docs = vectorstore.similarity_search(
    user_query,
    k=20,
    filter=ff
)

In [ ]:
print(len(retrieved_docs))
for i in retrieved_docs:
    print(i.metadata['start_date_int'])
    print(i.metadata['end_date_int'])
    # print(i)
    print('-'*20)

In [ ]:
# 리트리버로도 작동 되나 다시 테스트
# 리트리버 기본 세팅
retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={'k':10}
)

In [ ]:
retrieved_docs = retriever.invoke(
    user_query,
    search_kwargs={'filter': ff}
)

In [ ]:
print(len(retrieved_docs))
for i in retrieved_docs:
    # print(i.metadata['start_date_int'])
    # print(i.metadata['end_date_int'])
    print(i)
    print('-'*20)

In [ ]:
# 결과 : 적용 안 됨. 
# 리트리버로 구성 후 동적 필터링을 위해선..
# 1. 매 필터마다 동적으로 리트리버를 생성하거나(성능 문제에 대한 생각이 듦..)
# 2. configurableRetriever나 RunnableLambda 등을 사용해야 한다.
# => LCEL 공부의 필요성 ㅠ

# 일단 mvp니까.. 직접 호출하도록 구성하고, 후에 천천히 쌓아올리자.

In [ ]:
# 흠.. 클로드 좀 때려본 결과 다른 필터를 적용한 리트리버를 생성(as_retriever())하는 것은 
# 오버헤드가 그렇게 크지 않을 것으로 예상된다고 한다..
# 결국 난 랭체인을 활용해야하고, 그러기 위해서는 LCEL을 써야하며 LCEL을 쓰려면 단순 유사도 검색이 아니라
# retriever가 필요하므로.. 다른 문제가 발생하지 않는 한, as_retriever()에 동적 필터를 전달하는 방식을 사용하자


# 생각해 볼 것 : search_type mmr Vs similarity 



In [ ]:
# 테스트

def get_filtered_retriever(filter_condition):
    return vectorstore.as_retriever(
        search_type="mmr",
        search_kwargs={"k":10, "fetch_k":30, "lambda_mult":0.7, "filter": filter_condition}
    )

r = get_filtered_retriever(ff)

In [ ]:
d = r.invoke(user_query)


In [ ]:
print(len(d))
for i in d:
    print(i.metadata['start_date_int'])
    print(i.metadata['end_date_int'])
    # print(i)
    print('-'*20)